# Analyzing out-of-this world data, Part 2
Using data collected from the Open Exoplanet Catalogue database: https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue/

## Data License
Copyright (C) 2012 Hanno Rein

Permission is hereby granted, free of charge, to any person obtaining a copy of this database and associated scripts (the "Database"), to deal in the Database without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Database, and to permit persons to whom the Database is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Database. A reference to the Database shall be included in all scientific publications that make use of the Database.

THE DATABASE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE DATABASE OR THE USE OR OTHER DEALINGS IN THE DATABASE.

## Setup

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

planets = pd.read_csv('data/planets.csv')

Since we completed our EDA in the [`planets_ml.ipynb`](../lab_09/planets_ml.ipynb) notebook for last chapter, we will just look at the first 5 rows to refresh our memory of the data rather than repeating the EDA here.

In [ ]:
planets.head()

## Train Test Split

As in chapter 9, we will be predicting the length of a year (period) on our planets data:

In [ ]:
from sklearn.model_selection import train_test_split

data = planets[
    ['semimajoraxis', 'period', 'mass', 'eccentricity']
].dropna()
planets_X = data[['semimajoraxis', 'mass', 'eccentricity']]
planets_y = data.period

pl_X_train, pl_X_test, pl_y_train, pl_y_test = train_test_split(
    planets_X, planets_y, test_size=0.25, random_state=0
)

## Linear Regression to Predict Year Length from Lab 9
This was the result from chapter 9 for reference:

In [ ]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression().fit(pl_X_train, pl_y_train)
preds = lm.predict(pl_X_test)

Mean absolute error (MAE):

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(pl_y_test, preds)

### Hyperparameter Tuning with `GridSearchCV`
Note that `GridSearchCV` will try to maximize the scores, but we are using error metrics for all but $R^2$, so we use the versions marked with `neg_` as a prefix:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

model_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('lr', LinearRegression())
])

search_space = {
    'scale__with_mean': [True, False], 
    'scale__with_std': [True, False],
    'lr__fit_intercept': [True, False], 
    'lr__normalize': [True, False]
}

grid = GridSearchCV(
    model_pipeline, search_space, cv=5,
    scoring={
        'r_squared': 'r2', 
        'mse': 'neg_mean_squared_error', 
        'mae': 'neg_mean_absolute_error',
        'rmse': make_scorer(lambda x, y: -np.sqrt(mean_squared_error(x, y)))
    }, refit='mae'
).fit(pl_X_train, pl_y_train)

We can check the best score from the cross validation and the best hyperparameters in the search space:

In [ ]:
print('Best parameters (CV score=%.2f):\n%s' % (
    grid.best_score_, grid.best_params_
))

This reduced our MAE by over 120 Earth days (compared to chapter 9's model):

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(pl_y_test, grid.predict(pl_X_test))

## Can a decision tree tell us what features are important?

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(random_state=0).fit(pl_X_train, pl_y_train)
[(col, coef) for col, coef in zip(
    pl_X_train.columns, dt.feature_importances_
)]

We can visualize the decisions the tree is making:

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

graphviz.Source(export_graphviz(
    DecisionTreeRegressor(
        max_depth=4, random_state=0
    ).fit(pl_X_train, pl_y_train),
    feature_names=pl_X_train.columns
))

### Regularization
Ridge (L2) and LASSO (L1) are terms we can add to regression to penalize large coefficients. L2 uses squared coefficients and L1 uses the absolute value. LASSO tends to drive coeffiecients to zero and is therefore used for feature selection. We can combine the two in an elastic net. These help to reduce overfitting. Check the documentation for parameters that can be tuned:

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

ridge, lasso, elastic = Ridge(), Lasso(), ElasticNet()

for model in [ridge, lasso, elastic]:
    model.fit(pl_X_train, pl_y_train)
    print(
        f'{model.__class__.__name__}: '
        f'{model.score(pl_X_test, pl_y_test):.4}'
    )

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="../../lab_09/planets_ml.ipynb">
            <button>&#8592; Lab 9</button>
        </a>
        <a href="./red_wine.ipynb">
            <button>Red Wine</button>
        </a>
        <a href="./wine.ipynb">
            <button>Red + White Wine</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../solutions/lab_10/exercise_1.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../lab_11/1-EDA_unlabeled_data.ipynb">
            <button>Lab 11 &#8594;</button>
        </a>
    </div>
</div>
<hr>